# Recommendation System

In [6]:
#Libraries
import pandas as pd
import numpy as np
from random import *

Also we would like to mention that we prefered not to use the networkx library and create the recommendation methods with the standard python libraries. 
We know that networkx library could be a quicker implementation but we prefered to practice deeper on how the algorithms work.

Note:
In this file, the metadata are stored.

###  Datasets

#### Original Dataset

In [7]:
#Load the dataset
data=pd.read_csv("facebook_combined.txt",sep=" ", header=None)

#Add column names
data.columns = ["node1", "node2"]

#### Original dataset transformation

In [8]:
#Transform the graph to undirected
data2=pd.concat([data.node2,data.node1], axis=1)

#Rename the columns in order to merge the columns
data2.columns= ["node1", "node2"]
data=data.append(data2)

#Reset indexes
data = data.reset_index(drop=True)

#### Small test dataset

In [9]:
#Create a sample graph dataset
test_data = pd.DataFrame([[5, 2], 
                       [9, 3],
                       [9, 11],
                       [3, 6],
                       [4, 6],
                       [5, 7],
                       [1, 11],
                       [6, 2],
                       [7, 9],
                       [8, 9],
                       [5, 11],
                       [6, 7],
                       [6, 11],
                       [7, 6],
                       [2, 11],
                       [11,2],
                       [2, 5],
                       [2, 7],
                       [7, 2]],
                      columns=["node1", "node2"])

#### Test dataset friendships

In [10]:
# Take a look to the friendships  of test dataset
friendships={}
#Create friendships dict
for node in [1,2,3,4,5,6,7,8,9,11]:
    #Create a list with the friends of node
    ls=test_data[test_data.node1 == node]['node2'].tolist()

    #Create a dictionary with key the node and value the list
    friendships[node]=ls
    
print(friendships) 

{1: [11], 2: [11, 5, 7], 3: [6], 4: [6], 5: [2, 7, 11], 6: [2, 7, 11], 7: [9, 6, 2], 8: [9], 9: [3, 11], 11: [2]}


## Recommendation Methods

#### Recommending friends using Common neighbors (friend-of-friend (FoF) method)

In [87]:
##### Create the function for Common neighbors

def friendOfFriendScore(users, dataset, target):
    #Initialize
    l=list()
    friendships={}

    #Create friendships dict
    for node in users:
        #Create a list with the friends of node
        ls=dataset[dataset.node1 == node]['node2'].tolist()

        #Create a dictionary with key the node and value the list
        friendships[node]=ls

    # Initialize a dictionary with the intersections
    inter={}

    #Intersection between users
    for j in friendships:
        if (target != j) and (target not in friendships[j]) :
            intersection=(len(set(friendships.get(target)).intersection(set(friendships.get(j)))))
            
            #Keep intersection into a list
            inter[j]=intersection

    #Create a sorted list, in ties we take the smallest ID
    #We create tuples(id, score)
    lis=sorted(inter.items(), key=lambda value: value[1], reverse=True)
    

    #Final Result with only 10 elements
    return([(tup[0],round(tup[1],3)) for tup in lis[0:10]]);

In [88]:
##### Test the code for the Sample
users=[1,2,3,4,5,6,7,8,9,10,11]
friendOfFriendScore(users, test_data, 5)

[(6, 3), (1, 1), (7, 1), (9, 1), (11, 1), (3, 0), (4, 0), (8, 0), (10, 0)]

In [33]:
##### Test the code for the Original Dataset
users=list(range(0,4038))

#The output is a list with tuples (id, score)
print("The suggestions of 107 are:", friendOfFriendScore(users, data,107))
print("\nThe suggestions of 1126 are:", friendOfFriendScore(users, data,1126))
print("\nThe suggestions of 14 are:", friendOfFriendScore(users, data,14))
print("\nThe suggestions of 35 are:", friendOfFriendScore(users, data,35))

The suggestions of 107 are: [(513, 19), (400, 18), (559, 18), (373, 17), (492, 17), (500, 17), (378, 16), (436, 16), (431, 15), (514, 15)]

The suggestions of 1126 are: [(916, 113), (1238, 103), (1750, 99), (1230, 98), (1004, 94), (1791, 94), (1530, 90), (1172, 89), (1570, 85), (1597, 84)]

The suggestions of 14 are: [(2, 9), (17, 9), (140, 9), (111, 8), (137, 7), (162, 7), (19, 6), (333, 6), (44, 5), (243, 4)]

The suggestions of 35 are: [(46, 2), (68, 2), (99, 2), (131, 2), (175, 2), (177, 2), (225, 2), (227, 2), (278, 2), (321, 2)]


###  Recommending friends using Jaccard coefficient

In [78]:
##### Create the function for Common neighbors using Jaccard coefficient

def JaccardCoefficientScore(users, dataset, target):
    #Initialize
    l=list()
    friendships={}

    #Create friendships dict
    for node in users:
        #Create a list with the friends of node
        ls=dataset[dataset.node1 == node]['node2'].tolist()

        #Create a dictionary with key the node and value the list
        friendships[node]=ls

    # Initialize a dictionary with the intersections
    inter={}

    #Intersection between users
    for j in friendships:
        if (target != j) and (target not in friendships[j]) :
            
            # Create union
            union=len(set(friendships.get(target)).union(set(friendships.get(j))))
            
            # Check for No zero denominator
            if (union != 0) :
                inter[j]=len(set(friendships.get(target)).intersection(set(friendships.get(j))))/union

    #Create a sorted list, in ties we take the smallest ID
    #We create tuples(id, score)
    lis=sorted(inter.items(), key=lambda value: value[1], reverse=True)

    #Final Result
    return([(tup[0],round(tup[1],3)) for tup in lis[0:10]]);

In [79]:
##### Test the code for the Sample
users=[1,2,3,4,5,6,7,8,9,10,11]
JaccardCoefficientScore(users, test_data, 5)

[(6, 1.0),
 (1, 0.333),
 (11, 0.333),
 (9, 0.25),
 (7, 0.2),
 (3, 0.0),
 (4, 0.0),
 (8, 0.0),
 (10, 0.0)]

In [80]:
##### Test the code for the Original Dataset
users=list(range(0,4038))

print("The suggestions of 107 are:", JaccardCoefficientScore(users, data,107))
print("\nThe suggestions of 1126 are:", JaccardCoefficientScore(users, data,1126))
print("\nThe suggestions of 14 are:", JaccardCoefficientScore(users, data,14))
print("\nThe suggestions of 35 are:", JaccardCoefficientScore(users, data,35))

The suggestions of 107 are: [(513, 0.017), (400, 0.016), (559, 0.016), (492, 0.016), (500, 0.015), (373, 0.015), (436, 0.015), (378, 0.015), (515, 0.014), (514, 0.014)]

The suggestions of 1126 are: [(916, 0.487), (1750, 0.44), (1230, 0.436), (1530, 0.423), (1004, 0.42), (1238, 0.417), (1172, 0.401), (1791, 0.4), (1789, 0.374), (1597, 0.372)]

The suggestions of 14 are: [(2, 0.562), (140, 0.529), (17, 0.474), (162, 0.438), (111, 0.381), (333, 0.353), (44, 0.312), (137, 0.292), (19, 0.24), (243, 0.211)]

The suggestions of 35 are: [(321, 0.667), (11, 0.5), (12, 0.5), (15, 0.5), (18, 0.5), (37, 0.5), (43, 0.5), (74, 0.5), (114, 0.5), (209, 0.5)]


### Recommending friends using Adamic and Adar function

In [81]:
def AdamicAdarFunctionScore(users, dataset, target):
    #Initialize
    l=list()
    friendships={}

    #Create friendships dict
    for node in users:
        #Create a list with the friends of node
        ls=dataset[dataset.node1 == node]['node2'].tolist()

        #Create a dictionary with key the node and value the list
        friendships[node]=ls

    # Initialize a dictionary with the intersections
    inter={}

    #Intersection between users
    for j in friendships:
        if (target != j) and (target not in friendships[j]) :
            intersection = set(friendships.get(target)).intersection(set(friendships.get(j)))

            # Adamic and Adar score calculation
            sum = 0
            for k in intersection :
                if (k in friendships.keys()) and (friendships[k] != []) and len(friendships[k]) != 1:
                    sum = sum+1/np.log(len(friendships[k]))

            inter[j]=sum
   
    #Create a sorted list, in ties we take the smallest ID
    lis=sorted(inter.items(), key=lambda value: value[1], reverse=True)

    #Final Result
    return([(tup[0],round(tup[1],3)) for tup in lis[0:10]]);

In [82]:
##### Test the code for the Sample
users=[1,2,3,4,5,6,7,8,9,10,11]
AdamicAdarFunctionScore(users, test_data, 5)

[(6, 1.82),
 (7, 0.91),
 (11, 0.91),
 (1, 0),
 (3, 0),
 (4, 0),
 (8, 0),
 (9, 0),
 (10, 0)]

In [83]:
##### Test the code for the Original Dataset
users=list(range(0,4038))

print("The suggestions of 107 are:",AdamicAdarFunctionScore(users,data,107))
print("\nThe suggestions of 1126 are:",AdamicAdarFunctionScore(users,data,1126))
print("\nThe suggestions of 14 are:",AdamicAdarFunctionScore(users,data,14))
print("\nThe suggestions of 35 are:",AdamicAdarFunctionScore(users,data,35))

The suggestions of 107 are: [(513, 4.223), (400, 4.042), (559, 3.83), (500, 3.704), (492, 3.637), (373, 3.634), (378, 3.41), (436, 3.393), (524, 3.232), (514, 3.23)]

The suggestions of 1126 are: [(916, 23.259), (1238, 21.173), (1750, 20.193), (1230, 19.896), (1004, 19.127), (1791, 19.091), (1530, 18.157), (1172, 18.058), (1570, 17.375), (1597, 17.001)]

The suggestions of 14 are: [(2, 2.972), (17, 2.926), (140, 2.926), (111, 2.59), (162, 2.265), (137, 2.203), (333, 1.997), (19, 1.82), (44, 1.508), (243, 1.182)]

The suggestions of 35 are: [(46, 0.573), (68, 0.573), (99, 0.573), (131, 0.573), (175, 0.573), (177, 0.573), (225, 0.573), (227, 0.573), (278, 0.573), (321, 0.573)]


### Recommending Friends with Leicht-Holme-Newman Index (bonus method)

In [84]:
##### Create the function for Leicht Holme Newman method
def LeichtHolmeNewmanScore(users, dataset, target):
    #Initialize
    l=list()
    friendships={}

    #Create friendships dict
    for node in users:
        #Create a list with the friends of node
        ls=dataset[dataset.node1 == node]['node2'].tolist()

        #Create a dictionary with key the node and value the list
        friendships[node]=ls

    # Initialize a dictionary with the intersections
    inter={}

    #Intersection between users
    for j in friendships:
        if (target != j) and (target not in friendships[j]) :
            intersection=(len(set(friendships.get(target)).intersection(set(friendships.get(j)))))
            
            #Calculate the k for j and target
            k1=len(friendships.get(j))
            k2=len(friendships.get(target))
            
            if (k1 !=0 and k2 !=0):
                #Store the intersection in the list inter[]
                inter[j]=intersection/(k1*k2)
   
    #Create a sorted list, in ties we take the smallest ID
    lis=sorted(inter.items(), key=lambda value: value[1], reverse=True)

    #Final Result
    return([(tup[0],round(tup[1],3)) for tup in lis[0:10]]);

In [85]:
##### Test the code for the Sample
users=[1,2,3,4,5,6,7,8,9,10,11]
LeichtHolmeNewmanScore(users, test_data, 5)

[(1, 0.333),
 (6, 0.333),
 (11, 0.333),
 (9, 0.167),
 (7, 0.111),
 (3, 0.0),
 (4, 0.0),
 (8, 0.0)]

In [86]:
##### Test the code for the Original Dataset
users=list(range(0,4038))
print("The suggestions of 107 are:",LeichtHolmeNewmanScore(users, data,107))
print("\nThe suggestions of 1126 are:",LeichtHolmeNewmanScore(users, data,1126))
print("\nThe suggestions of 14 are:",LeichtHolmeNewmanScore(users, data,14))
print("\nThe suggestions of 35 are:",LeichtHolmeNewmanScore(users, data,35))

The suggestions of 107 are: [(11, 0.001), (12, 0.001), (15, 0.001), (18, 0.001), (37, 0.001), (43, 0.001), (74, 0.001), (114, 0.001), (209, 0.001), (210, 0.001)]

The suggestions of 1126 are: [(911, 0.005), (918, 0.005), (1096, 0.005), (1119, 0.005), (1145, 0.005), (1206, 0.005), (1262, 0.005), (1386, 0.005), (1395, 0.005), (1466, 0.005)]

The suggestions of 14 are: [(11, 0.067), (12, 0.067), (15, 0.067), (18, 0.067), (37, 0.067), (43, 0.067), (74, 0.067), (114, 0.067), (209, 0.067), (210, 0.067)]

The suggestions of 35 are: [(11, 0.5), (12, 0.5), (15, 0.5), (18, 0.5), (37, 0.5), (43, 0.5), (74, 0.5), (114, 0.5), (209, 0.5), (210, 0.5)]


## Evaluation of the recommendation system

### Algorithms Similarity

In [18]:
# Create users list
users=list(range(0,4038))

#Initialization
s1=[]
s2=[]
s3=[]

for i in list(range(100,4100,100)):
    
    #Run the functions
    fofList=friendOfFriendScore(users, data,i)
    JaccardList=JaccardCoefficientScore(users, data,i)
    AdamicAdarList=AdamicAdarFunctionScore(users, data,i)
    
    #Similarity Percentage of FoF and Jaccard
    s1.append(len(set(fofList).intersection(set(JaccardList)))*10)
    
    #Similarity Percentage of FoF and Adamic and Adar
    s2.append(len(set(fofList).intersection(set(AdamicAdarList)))*10)
    
    #Similarity Percentage of Jaccard and Adamic and Adar
    s3.append(len(set(AdamicAdarList).intersection(set(JaccardList)))*10)

#Average Similarity (%)
print("The average similarity of FoF & Jaccard is:",np.mean(s1),"%")
print("The average similarity of FoF & Adamic Adar is:",np.mean(s2),"%")
print("The average similarity of Adamic Adar & Jaccard is:",np.mean(s3),"%")

The average similarity of FoF & Jaccard is: 55.5 %
The average similarity of FoF & Adamic Adar is: 90.75 %
The average similarity of Adamic Adar & Jaccard is: 57.0 %


### Forecast Recommendations

#### Evaluation Function
In this stage we have to estimate the quality of the recommendation methods. We create a function(evaluationFunction()) which computes the strength of the connection between two nodes. In more details, we insert two already friends of our network and the function removes this connection from the dataset. After the connection drops, the algorythm searches for every method if one of the two nodes (ex. F1) exists in the list of the second node (ex. F2). We do the same process in both F1 and F2. Also, we would like to mention if a node doesn't exist in the recommendation list of the other node we exclude this relationship.

#### Calculation Score 
The score for each algorythm is calculated according to the position of the list.
Also, we take the average value of the position for both F1 and F2. The higher the score is, the higher the quality of the algorythm.


#### Create the functions for each method 

We create again the functions for each method but with the difference that these functions returns only a 10 element list with ids (without the id scores)

In [45]:
##### Create the function for Common neighbors
def friendOfFriend(users, dataset, target):
    #Initialize
    l=list()
    friendships={}

    #Create friendships dict
    for node in users:
        #Create a list with the friends of node
        ls=dataset[dataset.node1 == node]['node2'].tolist()

        #Create a dictionary with key the node and value the list
        friendships[node]=ls

    # Initialize a dictionary with the intersections
    inter={}

    #Intersection between users
    for j in friendships:
        if (target != j) and (target not in friendships[j]) :
            intersection=(len(set(friendships.get(target)).intersection(set(friendships.get(j)))))
            
            #Keep intersection into a list
            inter[j]=intersection
   
    #Create a sorted list, in ties we take the smallest ID
    lis=sorted(inter, key=inter.get, reverse=True)

    #Final Result
    return(lis[0:10]);

##### Create the function for Common neighbors using Jaccard coefficient
def JaccardCoefficient(users, dataset, target):
    #Initialize
    l=list()
    friendships={}

    #Create friendships dict
    for node in users:
        #Create a list with the friends of node
        ls=dataset[dataset.node1 == node]['node2'].tolist()

        #Create a dictionary with key the node and value the list
        friendships[node]=ls

    # Initialize a dictionary with the intersections
    inter={}

    #Intersection between users
    for j in friendships:
        if (target != j) and (target not in friendships[j]) :
            
            # Create union
            union=len(set(friendships.get(target)).union(set(friendships.get(j))))
            
            # Check for No zero denominator
            if (union != 0) :
                inter[j]=len(set(friendships.get(target)).intersection(set(friendships.get(j))))/union

    #Create a sorted list, in ties we take the smallest ID
    lis=sorted(inter, key=inter.get, reverse=True)

    #Final Result
    return(lis[0:10]);

##### Create the function for Common neighbors using Adamic Adar Method
def AdamicAdarFunction(users, dataset, target):
    #Initialize
    l=list()
    friendships={}

    #Create friendships dict
    for node in users:
        #Create a list with the friends of node
        ls=dataset[dataset.node1 == node]['node2'].tolist()

        #Create a dictionary with key the node and value the list
        friendships[node]=ls

    # Initialize a dictionary with the intersections
    inter={}

    #Intersection between users
    for j in friendships:
        if (target != j) and (target not in friendships[j]) :
            intersection = set(friendships.get(target)).intersection(set(friendships.get(j)))

            # Adamic and Adar score calculation
            sum = 0
            for k in intersection :
                if (k in friendships.keys()) and (friendships[k] != []) and len(friendships[k]) != 1:
                    sum = sum+1/np.log(len(friendships[k]))

            inter[j]=sum
   
    #Create a sorted list, in ties we take the smallest ID
    lis=sorted(inter, key=inter.get, reverse=True)

    #Final Result
    return(lis[0:10]);

#### Create the evaluation function

In [17]:
def evaluationFunction(dataset,users,F1,F2):

    ####Remove the relationship
    
    #First we find the connection F1-F2
    l1=dataset[dataset.node2 == F1].index
    l2=dataset[dataset.node1 == F2 ].index
    rm1=set(l1).intersection(set(l2))
    
    #Then we find the connection F2-F1
    l1=dataset[dataset.node2 == F2 ].index
    l2=dataset[dataset.node1 == F1].index
    rm2=set(l1).intersection(set(l2))

    #We create the union
    rm=rm1.union(rm2)
    
    #Remove the elements of the set rm
    for i in rm:
        dataset=dataset.drop(i)
    
    ###FoF (friend-of-friend)
    list1=friendOfFriend(users, dataset,F2)
    list2=friendOfFriend(users, dataset,F1)
    
    if (F1 in list1) and (F2 in list2): 
        
        #Compute the recommendations for F1
        Friend1=10 - list2.index(F2)

        #Compute the resommentdations for F2
        Friend2=10 - list1.index(F1)


        ####Compute the score
        scoreFoF=(Friend1+Friend2)/2
        
    else:
        return(None);
    
    ###Jaccard
    list1=JaccardCoefficient(users, dataset,F2)
    list2=JaccardCoefficient(users, dataset,F1)

    if (F1 in list1) and (F2 in list2): 
        
        #Compute the recommendations for F1
        Friend1=10 - list2.index(F2)

        #Compute the resommentdations for F2
        Friend2=10 - list1.index(F1)

        ####Check if either of these does not exist

        ####Compute the score
        scoreJaccard=(Friend1+Friend2)/2
        
    else:
        return(None);
    
    ###AdamicAdar
    list1=AdamicAdarFunction(users, dataset,F2)
    list2=AdamicAdarFunction(users, dataset,F1)
    
    if (F1 in list1) and (F2 in list2):
        
        #Compute the recommendations for F1
        Friend1=10 - AdamicAdarFunction(users, dataset,F1).index(F2)

        #Compute the resommentdations for F2
        Friend2=10 - AdamicAdarFunction(users, dataset,F2).index(F1)

        ####Check if either of these does not exist

        ####Compute the score
        scoreAdamicAdar=(Friend1+Friend2)/2
        
    else:
        return(None); 
          
 
    return(scoreFoF,scoreJaccard,scoreAdamicAdar);

#### Iteration Function

In order to have more accurate results we should run the algorythm more than once. So, we create the algorythm (finalscore()) in order to recall the evaluation function many times. Speciffically, this function takes a random index from the original dataset. After we call the evaluation function in order to create a score for each relationship; the outputs of the evaluation function are stored in a list. Finally, after all that loops, we calculate the average score for each method.

In [25]:
#Function for iterations
def finalScore(dataset,users, n):
    eval_scores=[]
    i=1
    while i <=n:
        
        #Random F1-F2 Relationship
        index=sample(range(0,4038),1)[0]
        friend1=dataset.iloc[index].node1
        friend2=dataset.iloc[index].node2
        
        evaluationOutput=evaluationFunction(dataset,users,friend1,friend2)
        
        if (evaluationOutput!= None):
            eval_scores.append(evaluationFunction(dataset,users,friend1,friend2))
            i += 1
        
    #Final Score
    if eval_scores != [] :
        scores=eval_scores      
        
    return(scores);       

In [28]:
users=list(range(0,4038))

#We create a variable in order to call once the function and then print the scores
score=finalScore(data,users,100)

#Create a score table
score_table=pd.DataFrame(score)

#Add column names
score_table.columns = ["FoF_Score", "Jaccard_Score", "Adamic_Adar_Score"]

# Final Print
print(score_table)

    FoF_Score  Jaccard_Score  Adamic_Adar_Score
0        10.0           10.0               10.0
1         4.5            6.5                5.0
2         8.5            8.0                8.5
3        10.0           10.0               10.0
4        10.0            6.5               10.0
5         9.5            8.0                9.5
6        10.0            9.5               10.0
7         9.0            9.5                9.0
8         7.0            5.0                7.0
9         9.5            9.5                9.5
10        9.5            9.0                9.5
11        4.5            7.5                8.0
12       10.0           10.0               10.0
13       10.0           10.0               10.0
14        9.5            9.5                9.5
15        7.0            4.0                6.5
16       10.0           10.0               10.0
17       10.0            9.5               10.0
18        8.0            7.0                8.0
19        7.0            6.5            

### Average Score

In [29]:
#Calculate the average Score
score1=score_table.FoF_Score.mean()
score2=score_table.Jaccard_Score.mean()
score3=score_table.Adamic_Adar_Score.mean()

#Final output
#The higher score is better because we have substracted the list position from 10
print("FoF score is: ", score1,"/10")
print("Jaccard score is: ", score2,"/10")
print("Adamic Adar score is: ", score3,"/10")

FoF score is:  9.025 /10
Jaccard score is:  8.78 /10
Adamic Adar score is:  9.085 /10


#### Comment the Output Score

As we can see the scoring of all functions is very close to each other but idependent of how many times we run the algorythm the sorting of the scores is the same. In other words, the Adamic Adar method has the higher score in all times, after Friend of Friend(FoF) method and after we have Jaccard method. In our view FoF method is a more generalized method that has decent results, if we take into consideration the calculation time. On the other hand the Adamic Adar method penalizes more the users that have a lot of friends than users that have less friends. This techique can better describes social network connections, and can have better results because is much closer on how people make a friend in a social network. The Jaccard distance is the less accurate metric for a reccomendation application.

## 2nd Evaluation System

We crete an alternative evaluation system in order to incease the accuracy.Specifficaly, we take a target node and we remove one by one all the friends of him. Hence, we calculate the number of the friends that suggested and then we divide it with the total initial number of friends. This computations are hard intensive, so we do this procedure for 2-3 different targets.

In [128]:
#Function for iterations
def finalScore2(dataset,users, n):
    
    eval_scores=[]
    
    #Create friendships dict
    friendships={}
    for node in users:
        #Create a list with the friends of node
        ls=dataset[dataset.node1 == node]['node2'].tolist()

        #Create a dictionary with key the node and value the list
        friendships[node]=ls

    for l in list(range(0,n)):
        
        target=sample(users,1)[0] #take 1 random user
        
        #Initialize the scores
        scorefof = 0
        scoreJaccard = 0
        scoreAdamicAdar = 0
        friends=[]
        
        for i in friendships.get(target):
            friends.append([target,i])
            
        for i in friends:
            evaluationOutput=evaluationFunction(dataset,users,i[0],i[1])

            if (evaluationOutput!= None):
                if evaluationOutput[0]!= 0:
                    scorefof +=1
                if evaluationOutput[1]!= 0:
                    scoreJaccard +=1
                if evaluationOutput[2]!= 0:    
                    scoreAdamicAdar +=1
        
        if len(friends) !=0 :
            eval_scores.append([scorefof/len(friends),scoreJaccard/len(friends),scoreAdamicAdar/len(friends)])
            
        #Final Print
        if eval_scores != [] :
            #Calculate the average value
            scores=np.mean(eval_scores,axis=0)
        
        
    return([round(scores[0],3),round(scores[1],3),round(scores[2],3)]);  

#### Final Score

In [132]:
#We create a variable in order to call once the function and then print the scores
score2=finalScore2(data,users,2)

print("FoF score is: ", score2[0],"/10")
print("Jaccard score is: ", score2[1],"/10")
print("Adamic Adar score is: ", score2[2],"/10")

FoF score is:  0.892 /10
Jaccard score is:  0.824 /10
Adamic Adar score is:  0.882 /10
